In [8]:
# necessary packages
import pandas as pd
from pandas.io.json import json_normalize
import gzip
import json
import multiprocessing as mp
import time
import copy
import pickle
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

## Creating a DataFrame from the Dataset

In [52]:
def convert_json(json_str):
#     converts json string to df
    return json_normalize(json.loads(json_str))

def convert_data(prefix, file_num):
    for i in range(file_num):
        suffix = str(i).zfill(12)
        filename = prefix + suffix
        
        with gzip.open(filename, "rt", encoding = "utf-8") as file:
            with mp.Pool() as pool:
                results = pool.map(convert_json, [line for line in file if line])
                mode = 'a' if i else 'w'
                pd.concat(results, sort=False).to_csv('data.csv', header = not i, mode = mode)        

In [ ]:
# convert gunzipped json files from google bigquery to a single csv
# convert_data('data/2019_01_04_', 4)

In [2]:
def load_data(filename):
    df = pd.read_csv(filename)
    df['outputs_addresses'] = df['outputs_addresses'].apply(
        lambda addr: addr[2:-2])
    df['inputs_addresses'] = df['inputs_addresses'].apply(
        lambda addr: addr[2:-2])
    return df
df = load_data('data.csv')

## Creating Address Clusters

In [9]:
def construct_clusters(df):
    clusters = []

    def cluster(group):
        addrs = group['inputs_addresses']
        new_clstr = True
        for idx, cluster in enumerate(clusters):
            for addr in addrs:
                if addr in cluster:
                    clusters[idx].update(addrs)
                    new_clstr = False
                    break
            if not new_clstr:
                break
        if new_clstr:
            clusters.append(set(addrs))

    start = time.time()
    df.groupby('tx_hash').progress_apply(cluster)
    end = time.time()
    print(f'This operation took {round((end-start) / 60, 2)} min')
    return clusters
# clusters = construct_clusters(df)


This operation took 864.24 min


In [13]:
def picklify(obj, filename):
# save obj in a pickle file for later
    with open(filename, 'wb') as file:
        pickle.dump(obj, file)
# picklify(clusters, 'clusters.pickle')

def unpicklify(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)
# unpicklify('clusters.pickle')

In [12]:
# inter_exists = False
# while true
# for cluster in clusters:
len(clusters)

99868

In [18]:
def clusters_are_consolidated(clusters):
    for idx, cluster in enumerate(clusters):
        for other in clusters[idx + 1:]:
            inter = cluster.intersection(other)
            if inter:
                return False
    return True

def consolidate_clusters(clusters):
    consolidated = clusters_are_consolidated(clusters)
    
    while()

found one
{'LXM85DT1SzANTercBXESNt2oNwb4uqKQ9N', 'LYTZsJ68h23GjzFaXHFCEyQ99iXGHh9ghF', 'LPJ9i67SPJWf7JmYeRxsFFugoVpZQtL4wF', 'LR6yCPA6Q8x5PomarybuVtjQ4gRebyQNF3', 'LLYQ4JUhkTmMJPSNv3Yts4G1UZaMzEvf9U', 'LhF6fSuHA65iJLHiDqDisU86vRXNSrKDR8', 'LZLrnngnPZmx5HPopWfnMrxADrvX2mhFAi', 'LgocYry3ZDKpPj6QfUPusr5CQ9ghj3DJji', 'LKgtwJJiz2sBwr8huyM6mYS6kreZBTcMNm', 'LahQC7gu3GDuUcCrivvZxSyorGoSgiTibv', 'LVkrzTpP1AWrVV4GF3dD39vB12ux9tWhUG', 'LSDsgZN4C3FDuy234187zjeNPHacagcpBN', 'LPi7BBeW5VBUKPCUbUpBtkGa1DDVwc5E8S', 'LbG4BrYyGZ1t9QGQDDHzBkHRku9eU66pzB', 'LR1twWX1YENzh1Ye2jcMFWHGptQcA7if1E', 'LVySz2iAGegLSF828kRPpp5crsZvfwvnKu', 'LTqyUnCiakmiDHNskw1Q97xamUSgWNTJog', 'LVvunWzoqWwP1MbnJyWMwcxKMQ1JFdDEox', 'LRkcFKXFGo2Csv5jy9VzoK9AmhVkhG7pUH', 'LTgWpoBCGwm5y7PWMNEShSLJy5sHo3TfyA', 'LbktzsvSmeGUTgxU67kPbKJ54mcYfnHv9J', 'LP1af5vfvu6wXxQYVpf29haiEFkFZkB7vb', 'LNFc4Shq5badNByjMcnt1DKPSh5gqGQVKA', 'LdxjerXWYJaTL17Qyv8pjoPSCaNREtx2DX', 'Lhdo65FdndWr34wWjkwt12kWF1wsfx61dy', 'LdqLoYwKoYMUz7eEjH5tRXSVFyRgkeHLzA', '

found one
{'Lg5r3zgU35i4Pv52FLgPFZciQX9RAKVYYL'}
found one
{'LfxV7eyLDpqdju9CuGV1H5NHCXagz6bqAB'}
found one
{'LhyLNfBkoKshT7R8Pce6vkB9T2cP2o84hx', 'Ld2EFMhha73VJcEaLJhAuiAHnsoMrCzPde', 'LX2zWe7o3gVmjwR8nKcmMVwL2wAJE2iLSS', 'LVq1N3fvawm5UPVQYKyCiaJE93NrhDHWWm', 'LZAYmu5FUuLq56jV5h3L4DLKadQ2a59iBc', 'LNDGR4egNkukmRZidFExftPb2YqjCaBnHV', 'LUqTBenon6UXr1AnynLH8zJC9cgnMbDwNy', 'LcwN74sxg1FqFsay4wmDCq8WScDN2c6Kqg', 'LMwzJgd5LQ8DoiMGRbi2LvXaVGDvgxQs51', 'LTyYrGCYTVKMnZYFxAWPVuNhmmaBdNo27z', 'LUgTv9FhDaXMymrtmh45HHReQALceY1s83', 'Leq4gYoo2CCuVaZiZgEpgrknrzyXAoqVMX', 'LMS7kYZgpA7sT5TAfMtFDJ6cmDtqzFJQVF', 'LTyufQXyuhCze84aAj4HQxV8jrMRRhNErP', 'LX8wY9192cDSWZwgTPXs9JjNp8khPyDhm6', 'LcpgskLs4e5Yfk5WJF5TYq7Loq5Mq22ut7', 'LPsLZhjvjq65eCgeSu5YJsxTScmC9MTC49', 'LLk1yEYVLFzYFfDJPCTBN1Yup4sMs8UdYs', 'LY82fLHArQXEErKvKu9VuZhNF8u9SfD2Yt', 'LL5SWfU9V7NujXmee9YerKhWKJoqi71KEp', 'LYSXaD2i34EEsWTb4r9BHjLSKgJWoVrwHW', 'LfWJFJj2TzikbAuno8KPBqegvtbirYkdZE', 'LbKe1Y17gxZBLFYmQKUNj2jJWRAFGs1TGM', 'LW3zHmcFrEcWDbZm

found one
{'MDWhK6x49tbNznAPA8wbCjNJ2bs8CySrDB'}
found one
{'MDWhK6x49tbNznAPA8wbCjNJ2bs8CySrDB', 'MMMRMkw7mPywdHGZpBFcwMBrH5vk1FmLkW'}
found one
{'Lf1ozwhQspwusGKd66jEmiodin794JoqZZ', 'MLPBVrLR7qwB5YJscsLqSsVucasrPz8LNo', 'LVGK791DvLA4NJQWVjTz8wetdyd8JaK7QS'}
found one
{'Lhfy5hDXBjrebFJa1QCEu3mxofcouLe2ir'}
found one
{'MLPBVrLR7qwB5YJscsLqSsVucasrPz8LNo'}
found one
{'LURi3fsWYmSNgTTRZ3wqrA3nWBE6hWwH9s'}
found one
{'MV5QaRxnGfQgvc7KJkYq7rvZRuv9o2mjAt'}
found one
{'LZqLHUCwFRWW1UXHWNVVdQFK1ywUp4YyxR'}
found one
{'LW4sJahzZi3Qp4nkkjqboTYqenc9ZBpbYv', 'LKfv3D5unv91pMXoYmR4aizpCsmfoUUSNs', 'LQmSdjyE9XFzuhrGd9mYz3w29fSguMcajd', 'Le5s3cL7syFfTYKu8k6dZ63t5PCVBmREsz', 'LLnVEbf59cWHnraj3PK9dsHbTKte7u48aa', 'LgVbSLvtQeLgvZM8uiwskKex38Qrdw9VQj', 'LKxjhrjy7hqwmMP8fxYaV6t1KNBia9XH3s', 'LZXF1JMtqJGZ8DQWbL1pm3GUVahVZur9SL', 'LMLyuV8jBswECKxaLHW3BEN7UyaQ58Lgz5', 'LSC1b2Rr2hGzyiFmnDFDfkHgJtqQuCvyzd', 'LTpUp9KzyYv6JrLeCUnQ6ykCPm4MXjQhLk', 'LiBYg6T35x4KqBaJoUNXVo7icyhnrFBjVK', 'LNycbb3FaamQbs6mQuWxZoQdkL

found one
{'MTB2fo3xPR8z8MafpYFLUch6v94rSqe6sh', 'LVjjfYcKDhYS3L8fFa14sWDTxcNhbhuvxp', 'LRWcYHR8zGT8dZY2a4UDSs9XhKKTrMNPM9', 'LiHzazEs36hWmGT4E3pySG5tuir2z8QBjn', 'MCKpWVor2m5w2mZUTWw69gwD6T6dHzSDeL', 'LZbKov7SwSbQookeCDVCchHTDGZ1vub4No', 'MW7WqedBU4aR95McetUZfL4vnNRr5Nh53y', 'LZLg1GtHkfKEPF3Vt6QaFSeUcF6FWCDg8P', 'Lc3BczAEVnBFeiq6w2EHy6UXMbQ3saiCAX', 'MGwkG6yjrkCSGonkmuLxjXLj6fJax3gAAR', 'LdU3CTHCqmbT9oZQRNCTQpPzGCMTtr5Pov', 'LaAPAmMM72VhTiW5dC8EJQEegFzuPfEiRV', 'LMP3WPQdvyNWcC4P67AuWt761xTxDMdPXL'}
found one
{'LcRQcieLFe9EN2PQ9kiWwLvJVepNFUxzTM', 'LSLwmkrwaDJ5R4xJyNcPZrY4nqvZ3Re19g', 'MB92ku8xV2jaefzWpLf1e4xzqz99UZF2EB', 'MLsghmkUcuPPPxC9irj5bvN4AzStNSmZFM'}
found one
{'LPG8AdnEyVJ47LJiM9t1KkKpZMCQNBSaCc'}
found one
{'LVkarNXYbYJpsXX6H4orZCxKmfNabovnXW'}
found one
{'LZLg1GtHkfKEPF3Vt6QaFSeUcF6FWCDg8P', 'LMP3WPQdvyNWcC4P67AuWt761xTxDMdPXL'}
found one
{'LbxhPD2Qg4BdK8LtbCXzjeUs7wToUNcbye', 'Lba7zSFEJPXYbSykCjPaDv27sz3RT6znoZ', 'LauUuN5G3wY52W9ceirwcD5WCWK1zPor7z', 'MBeyFeA86wB4WL7xGfJR9

found one
{'LbjGoKSbALhLV7GeqdFhwaAAqhALa4Xcxf', 'LcX6uRgeRtb98PXj5NQsb7CDULctcNkZjV'}
found one
{'MFWQcPf15mNcg55rtZqFFk5i4jniUZ9SGx'}
found one
{'LWV23wLpcuEaigDdzdRMzxS6NnVFKUtt9i'}
found one
{'LcX6uRgeRtb98PXj5NQsb7CDULctcNkZjV', 'LNLsmn4XoCJvYvrYaGzYXgyW8dHm3PtN47'}
found one
{'LWV23wLpcuEaigDdzdRMzxS6NnVFKUtt9i'}
found one
{'LeUuXD18SYhm3bCkD1z97oTjkaAquTzcqJ'}
found one
{'LRNJsahpC9otsRyiEaPLCUPGTXQiwuBeA7'}
found one
{'LT4Y9JpJR2F9BtA2XvECZVM99Kxn8RGzc9'}
found one
{'LTvgky1zcURwnt2YQ1BNjqmwMGfHJScYKc'}
found one
{'LMBEPDZRtvHsCjmXQG2gxXKooN4V9v4Lor'}
found one
{'LVxpoJLzSky5cDN8WtYBUqgfETthrZAvBU'}
found one
{'LT4Y9JpJR2F9BtA2XvECZVM99Kxn8RGzc9'}
found one
{'LNWn2gazF8cq8Se17UTbjUAzANhwKAqQyK'}
found one
{'MMKchh3ovvqmo8vW5N5Pdq7hXe6X8Ns262'}
found one
{'MVfUm5NRvyiXVpYXhKj4pVEDZe24VzP7gP'}
found one
{'nonstandard4d89e894ff93d4625c37f1b304d8e7d1dcd21770'}
found one
{'nonstandarde0980c1991fb1a3c0e98ea9dbfb2d8e393905c54'}
found one
{'nonstandard837eab1a6fa1cd5ddc6203c1e6ee1d0a9f

...

{'LZgZFJyTSe6RLZzajKzwdfQhmx77eUkLPQ'}
found one
{'LYkRSKCXik9aRxHMKbufJVtvwXMgrFdSbd'}
found one
{'LS8h28ttUvwwB7MDN6WzCDZXB6FGekEnLL'}
found one
{'LXukYoBjghxaNcD2KBBzGeFKpGL3kHnkLU'}
found one
{'LaGD7KvkqCSRGHJxLGNRqXja7k9pik25Bd'}
found one
{'Lh6Cvzr82SjsXqvYgfiDwRrWxiv8LLiQ8f'}
found one
{'LZve9PKjNRcn9nZKc4Rhbep5wz2L5W3F6y'}
found one
{'LKEuovbHVocirw8YJcs2pgVbbaZuvggazk'}
found one
{'LZmBBErwNXQY9J1QYQryBGxFpEWPoKoGYt'}
found one
{'LY7s1zo3bAmjsQFQct3UCFcgE7YMVc5aCi'}
found one
{'LZZAc6Z3P3UodLf1HPQLZzr2Zuqu9frTF7'}
found one
{'LcHdy8m1JJR1iWK5BhxaRgmbNyQNuRVnr6'}
found one
{'LVPZXenqYuWYCeo2sGEVeDyCakjGy1Zf7g'}
found one
{'LMFnF4obDDrRbjuqKah92HQgqKSPY4MdXV'}
found one
{'LYWjreUcEYthaTNBgVSqhprCsnCowN2bkx'}
found one
{'LMiCNhKWFYp5oUdA7wfuKKpVxfiXgWnFcD', 'MQa682pH4iUBPgB6J8wGV99tKaDcoN3LaT', 'MSP553TSx7hWkFakmDQkghwRHBu5gcDskq', 'Lb2Lt5m1HZsuxm9CHhpsUPRN9eKPZm4xZG', 'LSbxw184sKmadJ9ywSuWusamXkcU9zyr1v'}
found one
{'LNTM3rhMF1jopumbsMdRR3Kef4mJwPbgkt', 'LdVqutv816zndQcXVQDetdYv

found one
{'LKaGUsS17JGKnzdAP9XiaGhGKEzU9g6P84', 'LgWwvh9b62jzw91wTgpSzGDWM5ARGzX1Si', 'LgFPDmRC7JN7B3FkStoK5ALJGpzqY3ZQfv'}
found one
{'LgWwvh9b62jzw91wTgpSzGDWM5ARGzX1Si'}
found one
{'LgWwvh9b62jzw91wTgpSzGDWM5ARGzX1Si'}
found one
{'LgFPDmRC7JN7B3FkStoK5ALJGpzqY3ZQfv'}
found one
{'LKaGUsS17JGKnzdAP9XiaGhGKEzU9g6P84'}
found one
{'LgFPDmRC7JN7B3FkStoK5ALJGpzqY3ZQfv'}
found one
{'Lhop7W4AiHC4LRTKds53ty8KucymwKQhvZ'}
found one
{'LPfbPBuDeBWk1krcCYm4ze4zxANiiNHGHi'}
found one
{'LfYq78RwSbxLQgPP15Hyiz6JB2jyFmSAfb'}
found one
{'LYVMePEK3EfvcrkruTsieQNHNubAiJdqJH'}
found one
{'Laba7Gm2XgxNHGn2vboHAChueUygvrXubZ'}
found one
{'LTkGMSJFq1iYW6kk4LKkpyWWgqxXNWf4Mc'}
found one
{'MWoN24QVr5MRHhr1m4AVXEFBbmyA18hntX'}
found one
{'MJX5fL693mRxFE5Y5SsQnv1ZokoY1FaJC8'}
found one
{'MTNKmmA2ECWCq8hL9wrFgPw11cUst4iaPj'}
found one
{'MWoN24QVr5MRHhr1m4AVXEFBbmyA18hntX'}
found one
{'MTNKmmA2ECWCq8hL9wrFgPw11cUst4iaPj'}
found one
{'MGAcCnm4UGjojJXZRAY2Lm7pJfRs1bAWwB'}
found one
{'MVZr1Qj2xusYFxFCS9ZSFhokpomco4

{'LexMeqG7i394KdNdicshpC2GNvvxe23A9N'}
found one
{'Le8P6oPW6rWwNSD897ZsGP45yutXMzx7mn', 'Lccx9djsBUKdbox77cq39yNZL4HXCL3uga', 'LP3JFjUx1FG1z88zjoPZAorkvdRf87Szfe', 'LNDbAnb9KRt2vaJhkdtHqgZdjw9nJa75nW', 'LWbbos6jDndoc5MmmhQUKrgGAShWJFhke2', 'LPwjVfNo3aBE4pBBGPDyzx3VHA355CFuBk', 'LUZiR9h7kiaDZCZ7xzqdmBXpcvtjgUBrdK', 'LKktG3qrCJj87qN2t1TgThLATwRe8zWyNC', 'LSBFLuBsrne3k1MJNowoSLxqwCgPGGPaH1', 'LM2iCnw6bj9xCiCRiCGqVCJbBBx42jusGD', 'LMQ6u5nC5gxnmk2JocsaAKq6ZrKq9wKDQS', 'LesGP6a2F7iA2goQ6ruw41b58f6FDsTuUD', 'Lav2ToYpys7zg4C66b6U3hExuDMrtC7vGB', 'LW67zF3Ko7BzcdckQCsEQCRy2jrSJGszjF', 'LRZEXa8EqBsLbxvahBAKTcKbKe37XSmoYi', 'LUCXbZv62J2huuV9ZXUTCsb6C24W8Qts9z', 'Lcxg21Do3WfxYzLYTYNSPjmc5sKDHz5Es8', 'LY8PSqd3uvUjhmQTfeNisZegh28myVH4Uh'}
found one
{'LcnBL5e1LNzo8CSLqkdWsaj5WkqHe2LZYD'}
found one
{'LYgzC9hTkkQK84c4XboYzztgJ3VLqtGjZA'}
found one
{'LgsLMjRKtUZJYLX9wXtKCdrngv7nyXixi4'}
found one
{'LYUxoJSYPbgRqGwgPeCQGUkvQMUGvyUrxZ', 'LaHQ9XwHoGxBGzZrGJEPN5UiomkvgTvmg2'}
found one
{'LaHQ9XwHoGxBGzZrGJEP

found one
{'LejCxJy57ABmLyd1mseUvBYXFgUunXwtym'}
found one
{'La45hMDRrG2zixjzHZVsDQT1cZmaqLW58r'}


KeyboardInterrupt: 